In [10]:
import pandas as np

import psycopg2

from config import config

Connect to Database

In [11]:
def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)

        # create a cursor
        cur = conn.cursor()
    # execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')

        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            return conn

get tables names

In [15]:

conn = connect()
cur = conn.cursor()

t_query = """SELECT TABLE_NAME
                FROM INFORMATION_SCHEMA.TABLES
                WHERE TABLE_SCHEMA = 'public'"""
cur.execute(t_query)
names = [names[0] for names in cur.fetchall()]
print(names)

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 13.3 on x86_64-pc-linux-musl, compiled by gcc (Alpine 10.2.1_pre1) 10.2.1 20201203, 64-bit',)
['dic_units', 'dic_resource_names', 'dic_upper_works', 'dic_work_titles', 'json_projects', 'progress_ranges', 'res_progress_ranges', 'json_files', 'json_validation_files', 'json_validation_file_names', 'json_resources', 'json_validation_resources', 'json_works', 'json_validation_works']


Get Data To Numpy

In [16]:
query = "SELECT * from json_works"
source_df = np.read_sql_query(query, con=conn)
source_df.head(5)

C:\Users\Anatoly\AppData\Local\Temp\ipykernel_18364\541327484.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  source_df = np.read_sql_query(query, con=conn)


,rid,unit_name
0,1,шт
1,2,т
2,3,к-с
3,4,км
4,5,м2
